### Overview

#### Problem space: 
I am modelling it as a regression problem, where wine price is to be predicted based on wine features and reviews.
Overall objective is to demonstrate the comparative evaluation of different approaches and models. 


#### Dataset
Dataset available at Kaggle is much larger: https://www.kaggle.com/zynicide/wine-reviews

Using the file with 130K rows, and doing a 70:30 train-test split.

#### Approach: 
Pre-processing: Based on the exploratory analysis of dataset, pre-processing steps included:
1. Dropping rows with missing price values
2. Adding an additional column whose values are extracted from the values of an existing column
3. Replacing missing values with "none" and 0, for categorical and numerical columns respectively.

Demonstrated two approaches, with a neural network based regression model at the core. 

1. Baseline: Some feature engineering performed outside the neural network, which is a simple feed forward network with two dense hidden layers. Categorical features are transformed and encoded as n dimensional one hot vectors, resulting in very sparse and large feature vectors being fed to the neural network. Some variations are demonstrated here and results from 3 models with slightly different configurations are displayed. Models 1-3 below, network architecture displayed in the output cell. Further models can be trained using different combinations of provided feature extraction, and feature selection methods.


2. Improvement: Feature learning and regression included inside the neural network. Network inputs data where categorical features are encoded as numerical indexes in a pre-processing step, unlike large one hot encodings above. Dense vectors (embeddings) are learnt for each categorical feature within the network. These embeddings can also be extracted and fed to another regression algorithm. The feature with textual description of reviews is excluded in this approach, which can be included in more complex neural network architectures or network ensembles. Model 5 below, network architecture displayed in the output cell.

#### Evaluation:
Evaluation is performed on the test 
R2 score is used as the primary evaluation metrics. 
Best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected value of y, disregarding the input features, would get a R2 score of 0.0.
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html

#### Results:
Given limited experiments, approach 2 appears to be strongly outperforming approach 1

In [2]:
from platform import python_version
print(python_version())

2.7.14


In [4]:
import numpy as np
import sklearn
import tensorflow as tf

print(np.__version__)
print(sklearn.__version__)
print(tf.__version__)

1.13.1
0.19.1
1.14.2


In [5]:
import os
import pandas as pd
import pandas_profiling

In [ ]:
# change the path to your dataset location
wine_reviews = pd.read_csv("/Users/snegi/Downloads/wine-reviews/winemag-data-130k-v2.csv", 
                           index_col=0, encoding='utf-8')
wine_reviews.head(2)

In [6]:
wine_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129971 entries, 0 to 129970
Data columns (total 13 columns):
country                  129908 non-null object
description              129971 non-null object
designation              92506 non-null object
points                   129971 non-null int64
price                    120975 non-null float64
province                 129908 non-null object
region_1                 108724 non-null object
region_2                 50511 non-null object
taster_name              103727 non-null object
taster_twitter_handle    98758 non-null object
title                    129971 non-null object
variety                  129970 non-null object
winery                   129971 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 13.9+ MB


In [7]:
'''pandas profiling for exploratory analysis of dataset, like number of unique and null values in each column, correlation
between continuous valued columns, statistical summary (mean, median, standard deviation, quartiles etc.) 
of continuous valued columns'''
profile = pandas_profiling.ProfileReport(wine_reviews)
display(profile)


Number of variables,13
Number of observations,129971
Total Missing (%),0.0%
Total size in memory,18.9 MiB
Average record size in memory,152.0 B
Numeric,2
Categorical,11
Boolean,0
Date,0
Text (Unique),0
Rejected,0


#### Important observations from the above dataset profile :
- all columns except "point" are categorical and textual
- 'region_2' and 'designation' have very high number of missing values
- review points only range from 80 to 100, i.e only good reviews are available. This indicates that the available textual reviews ("description") are likely to have low variability in sentiments across different entries

### Variations of feature extraction

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer

In [8]:
'''one boolean-valued feature is constructed for each of the possible string values that the feature can take on. 
For instance, if title has a cardinality n, it will be expanded to n features in the output'''
# description columns can be ignored as it results in a very high dimensionality for the output feature vectors
def feature_extraction_1(df, description=False):
    if decription==False:
        df_X = df.drop(["price", "description"], axis=1)
    else:
        df_X = df.drop(["price"], axis=1)
    
    df_X_dict = df_X.to_dict("records")
    vec = DictVectorizer()
    X = vec.fit_transform(df_X_dict).toarray()
    print("Shape of data transformed into feature vectors:", X.shape)
    y = df["price"].values
    print("Size of prediction vector:", len(y))
    return(X, y)

In [9]:
# similar as above i.e. feature_extraction_1, but drops features with high missing values
def feature_extraction_2(df):
    df_X = df.drop(["price","region_2", "designation", "description"], axis=1)
    print("number of columns ", len(df_X.columns))
    df_X_dict = df_X.to_dict("records")
    vec = DictVectorizer()
    X = vec.fit_transform(df_X_dict).toarray()
    print("Shape of data transformed into feature vectors:", X.shape)
    y = df["price"].values
    print("Size of prediction vector:", len(y))
    return(X, y)

In [10]:
#excludes description feature
'''Uses a count based transformation for the title field, no. of transformed dimensions is equal to the vocabulary 
size of the text in the field'''
def feature_extraction_3(df):
    df_X = df.drop(["price", "region_2", "designation", "description"], axis=1)
    print("number of columns ", len(df_X.columns))
    df_X_1 = df_X[["title"]]
    df_X_2 = df_X.drop(["title"], axis=1)
    
    X_1_vectorizer = CountVectorizer()
    X_1 = X_1_vectorizer.fit_transform(df_X_1["title"].tolist()).toarray()
 
    df_X_2_dict = df_X_2.to_dict("records")
    vec = DictVectorizer()
    X_2 = vec.fit_transform(df_X_2_dict).toarray()
    
    X = np.concatenate((X_1,X_2),axis=1)
    
    print("Shape of data transformed into feature vectors:",X.shape)
    y = df["price"].values
    print("Size of prediction vector:",len(y))
    return(X,y)

In [33]:
'''Same as above, but includes description featur by transforming description field into vocabulary count based vector'''
def feature_extraction_4(df):
    df_X = df.drop(["price", "region_2", "designation", "twitter_handle"], axis=1)
    print("number of columns ", len(df_X.columns))
    df_X_1 = df_X[["title"]]
    df_X_2 = df_X[["description"]]
    df_X_3 = df_X.drop(["title", "description"], axis=1)
        
    X_1_vectorizer = CountVectorizer()
    X_1 = X_1_vectorizer.fit_transform(df_X_1["title"].tolist()).toarray()
 
    #top 100 tokens considered
    X_2_vectorizer = CountVectorizer(max_features=100)
    X_2 = X_2_vectorizer.fit_transform(df_X_2["description"].tolist()).toarray()

    df_X_3_dict = df_X_3.to_dict("records")
    vec = DictVectorizer()
    X_3 = vec.fit_transform(df_X_3_dict).toarray()
        
    X = np.concatenate((X_1, X_2, X_3),axis=1)
    
    print("Shape of data transformed into feature vectors:", X.shape)
    y = df["price"].values
    print("Size of prediction vector:", len(y))
    return(X, y)

### Feature selection and dimensionality reduction

In [12]:
from sklearn.decomposition import PCA

def dim_reduction(X, dimensions):
    pca = PCA(n_components=dimensions)
    X = pca.fit_transform(X)
    print("feature vector reduced to ", dimensions, " dimensions")
    return X

In [13]:
from sklearn.feature_selection import VarianceThreshold

def feature_selection(X, p=0.8):
    feature_sel = VarianceThreshold(threshold=(p * (1 - p)))
    X = feature_sel.fit_transform(X)
    print("no. of features selected: ", X.shape[1])
    return X

### NN model architecture

In [14]:
import tensorflow as tf
from tensorflow.keras.optimizers import *

def nn_model_fn(n_dimensions): 
    # a simple feed forward model architecture     
    batch_size = 32
    optimizer = 'adam'
    learning_rate = 0.01
    
    input_layer = tf.keras.layers.Input(shape=(n_dimensions,))
    Dense_layer_1 = tf.keras.layers.Dense(20, input_shape=(n_dimensions,))    
    Dense_layer_2 = tf.keras.layers.Dense(10)    
    Dense_layer_3 = tf.keras.layers.Dense(1, kernel_initializer = tf.initializers.glorot_uniform(seed=42))    
    logits = Dense_layer_2(Dense_layer_1(input_layer))
    model = tf.keras.models.Model(inputs=input_layer, outputs=logits)
    model.compile(loss='mse',
                  optimizer = optimizer,
                  metrics= ['mae', r2])
    model.summary()
    return model

In [120]:
# R2 score evaluation metrics often used for regression. Value ranges from -1 to +1
def r2(y_true, y_pred):
    SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return(1 - SS_res / (SS_tot + tf.keras.backend.epsilon()))

### Data Cleaning, pre-processing

In [17]:
# replacing missing values in string valued columns with the string "none"
def null_handling(df):
    values = {"country":"none", "designation":"none", "province":"none", "region_1":"none",
              "region_2":"none", "taster_name":"none", "taster_twitter_handle":"none", 
              "points":0,"title":"none", "variety":"none", "winery":"none"}
    df = df.fillna(value=values, inplace=False)
    return df

In [45]:
import re

# extracting year from the title 
def extract_year(row):
    year = re.search(r'([1-2][0-9]{3})',row['title'])
    if year is not None:
        year_value = year.group(1)
    else: year_value = "none"
    
    return year_value

def new_title(row):
    year = re.search(r'([1-2][0-9]{3})',row['title'])
    if year is not None:
        year_value = year.group(1)
        new_title = row['title'].replace(year_value,"")
    else: 
        year_value = "none"
        new_title = row['title']
    
    return new_title
 
# adding year as a separate column and replacing title column with a coulmn holding new title values which exclude year
def add_year_column(df):
    df['year'] = df.apply (lambda row: extract_year(row), axis=1)
    df['title'] = df.apply (lambda row: new_title(row), axis=1)
    
    return df

## Approach 1: Baseline
Simple Neural Network with  Feature Engineering and Feature Extraction

In [42]:
# since we are predicting price, dropping all the rows with missing values of price
df = wine_reviews.dropna(subset=["price"])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120975 entries, 1 to 129970
Data columns (total 13 columns):
country                  120916 non-null object
description              120975 non-null object
designation              86196 non-null object
points                   120975 non-null int64
price                    120975 non-null float64
province                 120916 non-null object
region_1                 101400 non-null object
region_2                 50292 non-null object
taster_name              96479 non-null object
taster_twitter_handle    91559 non-null object
title                    120975 non-null object
variety                  120974 non-null object
winery                   120975 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 12.9+ MB


### Model 1
Excludes description field. Doesn't use any dimensionality reduction or feature selection outside the neural net.

In [46]:
# additional column 'year' added
df_no_nulls = add_year_column(null_handling(df))

In [123]:
df_no_nulls.head(2)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,none,none,Roger Voss,@vossroger,Quinta dos Avidagos Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011
2,US,"Tart and snappy, the flavors of lime flesh and...",none,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013


In [51]:
df_no_nulls.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120975 entries, 1 to 129970
Data columns (total 14 columns):
country                  120975 non-null object
description              120975 non-null object
designation              120975 non-null object
points                   120975 non-null int64
price                    120975 non-null float64
province                 120975 non-null object
region_1                 120975 non-null object
region_2                 120975 non-null object
taster_name              120975 non-null object
taster_twitter_handle    120975 non-null object
title                    120975 non-null object
variety                  120975 non-null object
winery                   120975 non-null object
year                     120975 non-null object
dtypes: float64(1), int64(1), object(12)
memory usage: 13.8+ MB


In [124]:
# taking a smaller sample for faster execution
df_sample = df_no_nulls.sample(frac=.50, random_state=42)

In [140]:
(X, y) = feature_extraction_3(df_sample)

n_dimensions = X.shape[1]

('number of columns ', 10)
('Shape of data transformed into feature vectors:', (60488, 40801))
('Size of prediction vector:', 60488)


In [144]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=32)

In [128]:
model = nn_model_fn(n_dimensions)
model.fit(X_train, y_train, batch_size=32, shuffle=True, epochs=10, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 40801)             0         
_________________________________________________________________
dense_27 (Dense)             (None, 20)                816040    
_________________________________________________________________
dense_28 (Dense)             (None, 10)                210       
Total params: 816,250
Trainable params: 816,250
Non-trainable params: 0
_________________________________________________________________
Train on 33872 samples, validate on 8469 samples
Epoch 1/10
33872/33872 [==============================] - 61s 2ms/sample - loss: 1274.2957 - mean_absolute_error: 18.7348 - r2: -8.8154 - val_loss: 1168.3793 - val_mean_absolute_error: 15.8904 - val_r2: -6.6081
Epoch 2/10
33872/33872 [==============================] - 59s 2ms/sample - loss: 1027.8201 - mean_absolute_error: 15.0602 - r2: -5.7991 - va

In [129]:
model.evaluate(x=X_test, y=y_test, batch_size=32, verbose=1)

18147/18147 [==============================] - 15s 849us/sample - loss: 1201.5914 - mean_absolute_error: 13.9506 - r2: -5.2825


[1201.5914213105182, 13.950552, -5.2824736]

### Model 2
Same as above, includes description feature

In [132]:
# df_no_nulls = null_handling(df)
# df_sample = df_no_nulls.sample(frac =.50, random_state = 42)

# different feature extraction method
(X, y) = feature_extraction_4(df_sample)

('number of columns ', 11)
('Shape of data transformed into feature vectors:', (60488, 40901))
('Size of prediction vector:', 60488)


In [134]:
n_dimensions = X.shape[1]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=32)

model_2 = nn_model_fn(n_dimensions)
model_2.fit(X, y, batch_size=32, shuffle=True, epochs=10, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 40901)             0         
_________________________________________________________________
dense_30 (Dense)             (None, 20)                818040    
_________________________________________________________________
dense_31 (Dense)             (None, 10)                210       
Total params: 818,250
Trainable params: 818,250
Non-trainable params: 0
_________________________________________________________________
Train on 48390 samples, validate on 12098 samples
Epoch 1/10
48390/48390 [==============================] - 161s 3ms/sample - loss: 1263.3976 - mean_absolute_error: 17.0032 - r2: -7.4304 - val_loss: 1192.0377 - val_mean_absolute_error: 14.8183 - val_r2: -5.8256
Epoch 2/10
48390/48390 [==============================] - 110s 2ms/sample - loss: 1031.9111 - mean_absolute_error: 14.1157 - r2: -5.2103 -

In [54]:
model_2.evaluate(x=X_test, y=y_test, batch_size=32, verbose=1)

18147/18147 [==============================] - 16s 899us/sample - loss: 968.0836 - mean_absolute_error: 10.2335 - r2: -2.9665


[968.08356484722, 10.233475, -2.9665246]

### Model 3
Same as above, includes feature selection based on variance threshold

In [135]:
#X = dim_reduction(X, 2000)
(X, y) = feature_extraction_4(df_sample)
X = feature_selection(X, p=0.95)

('number of columns ', 11)
('Shape of data transformed into feature vectors:', (60488, 40901))
('Size of prediction vector:', 60488)
('no. of features selected: ', 148)


In [137]:
n_dimensions = X.shape[1]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=32)

model_3 = nn_model_fn(n_dimensions)
model_3.fit(X, y, batch_size=32, shuffle=True, epochs=10, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 148)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 20)                2980      
_________________________________________________________________
dense_34 (Dense)             (None, 10)                210       
Total params: 3,190
Trainable params: 3,190
Non-trainable params: 0
_________________________________________________________________
Train on 48390 samples, validate on 12098 samples
Epoch 1/10
48390/48390 [==============================] - 7s 147us/sample - loss: 1374.0738 - mean_absolute_error: 18.8496 - r2: -9.0173 - val_loss: 1356.5096 - val_mean_absolute_error: 15.9665 - val_r2: -7.1177
Epoch 2/10
48390/48390 [==============================] - 6s 131us/sample - loss: 1234.4092 - mean_absolute_error: 16.8742 - r2: -7.3260 - val

In [138]:
model_3.evaluate(x=X_test, y=y_test, batch_size=32, verbose=1)

18147/18147 [==============================] - 1s 73us/sample - loss: 1475.0477 - mean_absolute_error: 16.2734 - r2: -7.0093


[1475.047717614851, 16.273384, -7.009284]

Observations from the results above:
- Overall, low performance. Models do not fit well
- Including description based features improved the performance
- Feature selection outside the neural network produces poorer results. However, this can be specific to the simple feature selection method used here


There are several areas of improvements in the regression pipeline presented above:

1. Null replacement: nulls for continuous valued columns (points) can be replaced by interpolation
2. Feature engineering: use tokens instead of full string in categorical
3. Feature selection: 
4. Hyperparameter optimisation
5. Textual features: Description features can be incorparated using pre-trained word embeddings, text2vec, and ensemble methods of incorporating a 
6. Scalable implementation: An equivalent spark and sparkML imolementation would be better faster and scalable

## Approach 2: Improvement
Regression model learns dense representations for each categorical feature value within the network, without needing to input categorical features as one hot encodings.

In [71]:
'''The neural network architecture comprises of embedding layers corresponding to each categorical column. 
The dense categorical feature vectors from the embedding layer are concatenated with the numerical feature/s, 
followed by two dense layers.'''

from tensorflow.keras.layers import (Dense, Embedding,
                          Activation, Input, concatenate, Reshape, Flatten, PReLU)
from tensorflow.keras.models import Model


def nn_model_fn_embeddings(numerical_cols, emb_sizes):
        inputs = []
        flatten_layers = []

        for (var,sz) in emb_sizes.items():
            input_c = Input(shape=(1,), dtype='float32', name='input_' + var)
            embed_c = Embedding(*sz, input_length=1, name='embedded_' + var)(input_c)
            embed_c.trainable = True
            flatten_c = Flatten()(embed_c)

            inputs.append(input_c)
            flatten_layers.append(flatten_c)

        # concatenating the continuous-valued features
        input_num = Input(shape=(len(numerical_cols),), dtype='float32')
        flatten_layers.append(input_num)
        inputs.append(input_num)

        flatten = concatenate(flatten_layers, axis=-1)
        fc1 = Dense(200)(flatten)

        fc1 = Activation('relu')(fc1)
     

        fc2 = Dense(100, kernel_initializer='normal')(fc1)
        fc2 = Activation('relu')(fc2)
        
        
        output = Dense(1, kernel_initializer='normal', name='prediction_output')(fc2) 
        model = Model(inputs=inputs, outputs=output)
        
        model.compile(loss='mse',
                  optimizer = 'adam',
                  metrics= ['mae',r2])

        model.summary()
        return model
    
    
def prepare_inputs(X_df, embedding_sizes, numerical_cols):
    cat_vars = embedding_sizes.keys()
    #numerical_vars = [x for x in X.columns if x not in cat_vars]
    x_inputs = []
        
    for col in cat_vars:
            x_inputs.append(X_df[col].values)

    x_inputs.append(X_df[numerical_cols].values)
    
    return x_inputs


def calc_embedding_size(df):
    categorical_cols = [x for x in df.columns if df[x].dtype == object]
    df_cat = df[categorical_cols]
    cat_sizes = df_cat.apply(pd.Series.nunique).to_dict()  
    
            
    col_emb_sizes={col: (cat_sizes[col],np.minimum(50, (cat_sizes[col] + 1) // 2))
                  for col in cat_sizes.keys()}
    
    return col_emb_sizes

In [78]:
df = wine_reviews.dropna(subset=["price"])
df_no_nulls = add_year_column(null_handling(df))
df_sample = df_no_nulls.sample(frac =.50, random_state = 42)

In [80]:
X_df = df_sample.drop(["description", "region_2", "designation","price"],axis=1)


numerical_cols = [x for x in X_df.columns if X_df[x].dtype != object]
embedding_sizes = calc_embedding_size(X_df)

In [81]:
from sklearn.preprocessing import LabelEncoder

#encodes string valued features as numerical indexes and normalises numeric features
X_df = X_df.apply(LabelEncoder().fit_transform)
y_df = df_sample["price"]


X_df_train, X_df_test = train_test_split(X_df, test_size=0.3, random_state=32)
y_df_train, y_df_test = train_test_split(y_df, test_size=0.3, random_state=32)


X_train_inputs_list = prepare_inputs(X_df_train, embedding_sizes, numerical_cols)
X_test_inputs_list = prepare_inputs(X_df_test, embedding_sizes, numerical_cols)

### Model 5

In [83]:
model_5 = nn_model_fn_embeddings(numerical_cols,embedding_sizes)
model_5.fit(X_train_inputs_list, y_df_train, batch_size=32, epochs=10, validation_split=0.2)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_province (InputLayer)     (None, 1)            0                                            
__________________________________________________________________________________________________
input_variety (InputLayer)      (None, 1)            0                                            
__________________________________________________________________________________________________
input_region_1 (InputLayer)     (None, 1)            0                                            
__________________________________________________________________________________________________
input_country (InputLayer)      (None, 1)            0                                            
__________________________________________________________________________________________________
input_tast

In [121]:
from sklearn.metrics import r2_score

y_pred = model_5.predict(X_test_inputs_list)
y_pred = np.reshape(y_pred, (y_pred.shape)[0])
y_test = np.array(y_df_test.tolist())
r2_score(y_test, y_pred)

0.47614041042577204